In [1]:
from __future__ import division
from IPython.display import display
from IPython.display import Markdown as md
from IPython.core.display import SVG
from sympy.interactive import printing
printing.init_printing(use_latex='mathjax')

import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
from scipy.odr import *
import scipy.odr.odrpack as odrpack
from uncertainties import ufloat
from uncertainties.umath import *
from uncertainties import unumpy

import sympy as sp
sf = sp.sympify

import scipy.constants as cm
import pint as pn
ureg = pn.UnitRegistry(system='mks')
ureg.default_format = '~P'
pe = ureg.parse_expression

## Konstanten und Einheiten
CM = pd.DataFrame(cm.physical_constants).T
CM['c'] = CM.index
CM = CM.reset_index(drop=True)
def getpc(quant):
    DF = CM
    DF = DF[DF['c']==quant].reset_index(drop=True)
    quan = DF[0][0] * ureg(DF[1][0])
    return quan
def addnoise(X,scale=0.01,lin=False):
    if not lin:
        mag = scale*(np.max(X.m)-np.min(X.m))
        return np.random.normal(0,mag,len(X.m))*X.u
    if lin:
        return np.random.normal(0,1,len(X.m))*X*scale
def uODR(func,beta0,xdata,ydata,**kwargs):
    xfit = kwargs.get('xfit',xdata)
    sx = kwargs.get('sx',None)
    sy = kwargs.get('sy',None)
    model = Model(func)
    data = RealData(xdata,ydata, sx=sx, sy=sy)
    odr = ODR(data, model, beta0=beta0)
    out = odr.run()
    # Estimated parameter values, of shape q
    p = out.beta
    # Standard errors of the estimated parameters, of shape p
    perr = out.sd_beta
    up =unumpy.uarray(p, perr)
    yfit=func(p, xfit) 
    return up , p, yfit, out

In [2]:
g = getpc('standard acceleration of gravity')
m_e = getpc('electron mass')
e = getpc('elementary charge')
c = getpc('speed of light in vacuum')
e0 = getpc('electric constant')
mu0 = getpc('mag. constant')

In [3]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Quellcode an/aus schalten."></form>''')

# Energie von Photonen und Elektronen


### Der photoelektrische Effekt
(ausführen evtl. nur vom PC möglich)

In [4]:
from IPython.core.display import display, HTML
display(HTML('<div style="position: relative; width: 300px; height: 197px;"><a href="https://phet.colorado.edu/sims/photoelectric/photoelectric_de.jar" style="text-decoration: none;"><img src="https://phet.colorado.edu/sims/photoelectric/photoelectric-600.png" alt="Der photoelektrische Effekt" style="border: none;" width="300" height="197"/><div style="position: absolute; width: 200px; height: 80px; left: 50px; top: 58px; background-color: #FFF; opacity: 0.6; filter: alpha(opacity = 60);"></div><table style="position: absolute; width: 200px; height: 80px; left: 50px; top: 58px;"><tr><td style="text-align: center; color: #000; font-size: 24px; font-family: Arial,sans-serif;">Zum Starten klicken</td></tr></table></a></div>'))

Zum Starten klicken


### Schwarzkörper Strahlung

In [5]:
from IPython.display import IFrame

IFrame(src='https://phet.colorado.edu/sims/html/blackbody-spectrum/latest/blackbody-spectrum_de.html', width=700, height=500)

### Bragg-Reflexion

In [6]:
from IPython.display import IFrame

IFrame(src='./material/bragg.html', width=700, height=500)